# Download Data

In [1]:
from grabData import fetch_housing_data

In [2]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = "datasets/housing"
HOUSING_URL = DOWNLOAD_ROOT + HOUSING_PATH + "/housing.tgz"

In [3]:
fetch_housing_data(HOUSING_URL, HOUSING_PATH)

TypeError: fetch_housing_data() got an unexpected keyword argument 'housing_url'